---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def graph_identification():
    
    l=[]
    for i in P1_Graphs:
        s1 = nx.average_clustering(i)
        s2 = nx.average_shortest_path_length(i)
        s3=[]
        ns=len(nx.nodes(i))
        degs=list(nx.degree(i).values())
        uni=sorted(set(degs))
        for j in uni:
            s3.append(degs.count(j))
        gr=''
        if len(s3)>15:
            gr='PA'
        elif s1>0.1:
            gr='SW_L'
        else:
            gr='SW_H'
        l.append(gr)
    return l

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [4]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [5]:
def salary_predictions():
    
    nodes= G.nodes(data=True)
    idx,dep,sal=[],[],[]
    for i in nodes:
        idx.append(i[0])
        dep.append(i[1]['Department'])
        sal.append(i[1]['ManagementSalary'])
    
    df= pd.DataFrame({'Department':dep, 'ManagementSalary':sal},index=idx)
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())
    df['degree_centrality'] = pd.Series(nx.degree_centrality(G))
    df['closeness'] = pd.Series(nx.closeness_centrality(G, normalized=True))
    df['betweeness'] = pd.Series(nx.betweenness_centrality(G, normalized=True))
    df['pr'] = pd.Series(nx.pagerank(G))
    df2 = (df[np.isnan(df['ManagementSalary'])])
    df2.drop(['ManagementSalary'],axis=1,inplace=True)
    df.dropna(inplace=True)
    
    X_train,y_train, X_test= df.drop(['ManagementSalary'],axis=1), (df['ManagementSalary']).reshape(-1,1), df2
    
    scaler= MinMaxScaler().fit(X_train)
    X_tr = scaler.transform(X_train)
    X_te = scaler.transform(X_test)
    
    clf = SVC(C=3, probability=True).fit(X_tr, y_train)
    return pd.Series(clf.predict_proba(X_te)[:,1], index=df2.index)

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [6]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [8]:
def new_connections_predictions():
        
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.ensemble import RandomForestClassifier
    
    
    future_connections['Preferential Attachment'] = [i[2] for i in nx.preferential_attachment(G, future_connections.index)]
    future_connections['Common Neighbours'] = [len(list(nx.common_neighbors(G, x[0], x[1]))) for x in future_connections.index]
    future_connections['Jaccard Coefficient'] = [i[2] for i in nx.jaccard_coefficient(G, future_connections.index)]
    future_connections['Resource Allocation'] = [i[2] for i in nx.resource_allocation_index(G, future_connections.index)]
    future_connections['Adamic Adar'] = [i[2] for i in nx.adamic_adar_index(G, future_connections.index)]
    
    for node in G.nodes():
        G.node[node]['community'] = G.node[node]['Department']
    future_connections['cn_soundarajan_hopcroft'] = [i[2] for i in nx.cn_soundarajan_hopcroft(G, future_connections.index)]   
    future_connections['ra_soundarajan_hopcroft'] = [i[2] for i in nx.ra_index_soundarajan_hopcroft(G, future_connections.index)]
    
    
    Train_df = future_connections[~pd.isnull(future_connections['Future Connection'])]
    Test_df = future_connections[pd.isnull(future_connections['Future Connection'])]
    features =['Preferential Attachment', 'Common Neighbours',
       'Jaccard Coefficient', 'Resource Allocation', 'Adamic Adar',
       'cn_soundarajan_hopcroft', 'ra_soundarajan_hopcroft']
    X_train = Train_df[features]
    y_train = Train_df['Future Connection']
    X_test = Test_df[features]
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    clf = RandomForestClassifier(n_estimators=100, max_depth=10).fit(X_train_scaled, y_train)
    predictions = (clf.predict_proba(X_test_scaled)[:,1]).tolist()
    return pd.Series(predictions,X_test.index)

(107, 348)    0.028278
(542, 751)    0.012534
(20, 426)     0.538988
(50, 989)     0.012518
(942, 986)    0.012707
(324, 857)    0.012518
(13, 710)     0.137760
(19, 271)     0.149423
(319, 878)    0.012506
(659, 707)    0.012516
(49, 843)     0.012531
(208, 893)    0.012557
(377, 469)    0.014067
(405, 999)    0.015095
(129, 740)    0.013424
(292, 618)    0.018584
(239, 689)    0.012376
(359, 373)    0.017712
(53, 523)     0.407284
(276, 984)    0.012183
(202, 997)    0.012506
(604, 619)    0.040921
(270, 911)    0.012430
(261, 481)    0.061827
(200, 450)    0.991071
(213, 634)    0.012539
(644, 735)    0.041114
(346, 553)    0.012455
(521, 738)    0.012680
(422, 953)    0.014213
                ...   
(672, 848)    0.012430
(28, 127)     0.980508
(202, 661)    0.012402
(54, 195)     0.999978
(295, 864)    0.012550
(814, 936)    0.012443
(839, 874)    0.012707
(139, 843)    0.012518
(461, 544)    0.013909
(68, 487)     0.013074
(622, 932)    0.012621
(504, 936)    0.016995
(479, 528) 